#### Rats in the City Pt. 3 - Choropleth Map

---

Choropleth maps are an excellent way to showcase geographical data so that is what we are going to do with the rat sightings data. 

---

*Libraries*

In [1]:
import pandas as pd
import numpy as np

from bokeh.io import curdoc

import yaml

from bokeh.io import (
    output_notebook,
    show
)

from bokeh.plotting import (
    Column,
    figure,
    gridplot,
    Row
)

from bokeh.themes import Theme
from bokeh.layouts import widgetbox
from bokeh.palettes import Blues9, OrRd9
from bokeh.transform import jitter
from bokeh.tile_providers import CARTODBPOSITRON

from bokeh.models import (
    BasicTicker, ColumnDataSource, ColorBar,
    ColorMapper, CustomJS, Div,
    HBar, HoverTool, LinearColorMapper,
    LogColorMapper, MultiSelect, PrintfTickFormatter,
    Select
)

output_notebook()

Loading BokehJS ...

---

#### Load Data

In [2]:
df = pd.read_pickle("./data/Rat_Sightings.pkl")
df.head()

,Created Date,Location Type,postalCode,Location,Borough,poly_x,poly_y,xs,ys,boro_poly_x,...,boro_xs,boro_ys,Neighborhood,nhood_poly_x,nhood_poly_y,nhood_xs,nhood_ys,Year,Season,Month
0,08/02/2017 12:00:00 AM,3+ Family Mixed Use Building,10001,"(40.7456163166638, -73.98757087477563)",Manhattan,"[-74.00827017711134, -74.00762507843844, -74.0...","[40.752587143684494, 40.75427918134466, 40.756...","[-8238562.950607049, -8238491.138551269, -8238...","[4975916.533234529, 4976165.180545851, 4976422...","[-74.01092841268033, -74.01193259911061, -74.0...",...,"[-8238858.864036998, -8238970.649559078, -8238...","[4965915.024392721, 4965826.398628179, 4965856...",Midtown,"[-73.9622312516561, -73.96791907791587, -73.97...","[40.75452213042196, 40.747960560355295, 40.752...","[-8233437.920868702, -8234071.086791659, -8235...","[4976200.882773609, 4975236.682617902, 4975947...",2017,summer,Aug
1,09/08/2016 12:00:00 AM,Commercial Building,10001,"(40.74864397036016, -73.98954087053794)",Manhattan,"[-74.00827017711134, -74.00762507843844, -74.0...","[40.752587143684494, 40.75427918134466, 40.756...","[-8238562.950607049, -8238491.138551269, -8238...","[4975916.533234529, 4976165.180545851, 4976422...","[-74.01092841268033, -74.01193259911061, -74.0...",...,"[-8238858.864036998, -8238970.649559078, -8238...","[4965915.024392721, 4965826.398628179, 4965856...",Midtown,"[-73.9622312516561, -73.96791907791587, -73.97...","[40.75452213042196, 40.747960560355295, 40.752...","[-8233437.920868702, -8234071.086791659, -8235...","[4976200.882773609, 4975236.682617902, 4975947...",2016,summer,Sep
2,01/12/2016 12:00:00 AM,3+ Family Apt. Building,10001,"(40.74925101726878, -74.00199584240312)",Manhattan,"[-74.00827017711134, -74.00762507843844, -74.0...","[40.752587143684494, 40.75427918134466, 40.756...","[-8238562.950607049, -8238491.138551269, -8238...","[4975916.533234529, 4976165.180545851, 4976422...","[-74.01092841268033, -74.01193259911061, -74.0...",...,"[-8238858.864036998, -8238970.649559078, -8238...","[4965915.024392721, 4965826.398628179, 4965856...",Midtown,"[-73.9622312516561, -73.96791907791587, -73.97...","[40.75452213042196, 40.747960560355295, 40.752...","[-8233437.920868702, -8234071.086791659, -8235...","[4976200.882773609, 4975236.682617902, 4975947...",2016,winter,Jan
3,07/14/2017 12:00:00 AM,Construction Site,10001,"(40.745421892217756, -73.9929157080262)",Manhattan,"[-74.00827017711134, -74.00762507843844, -74.0...","[40.752587143684494, 40.75427918134466, 40.756...","[-8238562.950607049, -8238491.138551269, -8238...","[4975916.533234529, 4976165.180545851, 4976422...","[-74.01092841268033, -74.01193259911061, -74.0...",...,"[-8238858.864036998, -8238970.649559078, -8238...","[4965915.024392721, 4965826.398628179, 4965856...",Midtown,"[-73.9622312516561, -73.96791907791587, -73.97...","[40.75452213042196, 40.747960560355295, 40.752...","[-8233437.920868702, -8234071.086791659, -8235...","[4976200.882773609, 4975236.682617902, 4975947...",2017,summer,July
4,06/15/2017 12:00:00 AM,Vacant Lot,10001,"(40.74518584464681, -73.99293016871866)",Manhattan,"[-74.00827017711134, -74.00762507843844, -74.0...","[40.752587143684494, 40.75427918134466, 40.756...","[-8238562.950607049, -8238491.138551269, -8238...","[4975916.533234529, 4976165.180545851, 4976422...","[-74.01092841268033, -74.01193259911061, -74.0...",...,"[-8238858.864036998, -8238970.649559078, -8238...","[4965915.024392721, 4965826.398628179, 4965856...",Midtown,"[-73.9622312516561, -73.96791907791587, -73.97...","[40.75452213042196, 40.747960560355295, 40.752...","[-8233437.920868702, -8234071.086791659, -8235...","[4976200.882773609, 4975236.682617902, 4975947...",2017,spring,June


---

#### Create Choropleth Map

In [3]:
def choro_map(doc):
    
    df = pd.read_pickle("./data/Rat_Sightings.pkl")

    def make_map(var="postalCode", year="All", season="All"):
        
        def filter_years_seasons(df=df, season=season, year=year):
    
            if year == "All" and season =="All":

                #group dataframe by indicated column name and rename columns
                df = (df
                            )

            elif year != "All" and season == "All":

                #group dataframe by indicated column name and rename columns
                df = (df
                      .query("Year == '%s'" %year)
                     )

            elif year != "All" and season != "All":

                # select cases for user-selected year and season
                df = (df
                     .query("Year == '%s'" %year)
                     .query("Season == '%s'" %season)
                    )

            elif year == "All" and season != "All":

                # only extract cases for the user-selected season
                df = (df
                      .query("Season == '%s'" %season)    
                     )

            else:

                # just set it to itself if user-error occurs
                df = df   


            return df
        
        new_df = filter_years_seasons()

        if var=="postalCode":

            groupeddf = (new_df
                         .assign(n=0)
                         .groupby(var)
                         .n
                         .count()
                         .reset_index()
                         .rename(columns = {"n":"Rat_Sightings"})
                         .merge(df[["postalCode", "Neighborhood",
                                    "Borough", "xs", "ys"
                                   ]])
                         .drop_duplicates(subset=[var])
                         .reset_index(drop=True)
                        )

        elif var == "Neighborhood":

            groupeddf = (new_df
                         .assign(n=0)
                         .groupby(var)
                         .n
                         .count()
                         .reset_index()
                         .rename(columns = {"n":"Rat_Sightings"})
                         .merge(df[["Neighborhood", "Borough",
                                    "nhood_xs", "nhood_ys"
                                   ]])
                         .drop_duplicates(subset=[var])
                         .reset_index(drop=True)
                         .rename(columns={"nhood_xs":"xs", "nhood_ys":"ys"})
                        )

        elif var == "Borough":
            groupeddf = (new_df
                         .assign(n=0)
                         .groupby(var)
                         .n
                         .count()
                         .reset_index()
                         .rename(columns = {"n":"Rat_Sightings"})
                         .merge(df[["Borough", 
                                    "boro_xs", "boro_ys"
                                   ]])
                         .drop_duplicates(subset=[var])
                         .reset_index(drop=True)
                         .rename(columns={"boro_xs":"xs", "boro_ys":"ys"})
                        )
        else:
            
            var = "postalCode"

            groupeddf = (new_df
                         .assign(n=0)
                         .groupby(var)
                         .n
                         .count()
                         .reset_index()
                         .rename(columns = {"n":"Rat_Sightings"})
                         .merge(df[["postalCode", "Neighborhood",
                                    "Borough", "xs", "ys"
                                   ]])
                         .drop_duplicates(subset=[var])
                         .reset_index(drop=True)
                        )


        # instantiate the color mapper
        color_mapper = LogColorMapper(
            palette=OrRd9[::-1]
        )

        p = figure(
            x_range=(-8400000,-8100000),
            y_range=(4950000, 5000000),
            x_axis_type = "mercator",
            y_axis_type="mercator",
            plot_height=1200,
            plot_width=1000
        )
        p.axis.visible = False
        p.grid.grid_line_color = None
        p.add_tile(CARTODBPOSITRON)

        p.grid.grid_line_color = None

        p.patches(
            "xs",
            "ys",
            source=ColumnDataSource(data=groupeddf),
            fill_color = {
                "field":"Rat_Sightings",
                "transform":color_mapper
            },
            fill_alpha=0.6
        )
        
        p.add_tools(HoverTool(tooltips=[
            ("Number of Rat Sightings: ", "@Rat_Sightings"),
            ("%s" %var, "@%s" %var)
        ]))

        return p

    year_select = Select(value="All",
                         options=[*df["Year"].unique()]+["All"],
                         title="Select a year to view: "
                        )
    
    season_select = Select(value="All", 
                           options=[*df["Season"].unique()]+["All"],
                           title="Select a season to view: "
                          )
    
    agg_level_select = Select(value="postalCode",
                              options=["postalCode", "Neighborhood",
                                       "Borough"
                                      ],
                              title="Select a value to view the data by: "
                             )


    def update_plot(attr, old, new):

        layout.children[1] = make_map(year=year_select.value,
                                      season=season_select.value,
                                      var=agg_level_select.value
                        
                                     )

    year_select.on_change("value", update_plot)
    season_select.on_change("value", update_plot)
    agg_level_select.on_change("value", update_plot)

    layout = Column(Column(agg_level_select,
                           year_select,
                           season_select
                          ),
                    make_map()
                   )

    doc.add_root(layout)

    doc.theme = Theme(json=yaml.load(
            """ attrs:
                    Figure:
                        background_fill_color: "#DDDDDD"
                        outline_line_color: white
                        toolbar_location: above
                        height: 500
                        width: 800
            """
        ))
    

show(choro_map)